In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [70]:
import pandas as pd
import numpy as np
import math as mt

from src import *

# 1. Load CSV

In [27]:
data = pd.read_csv("data/data.csv")
data.drop('uid', inplace=True, axis=1)
data = data[data['ttime'].map(lambda x: "2017" in x)]

print("datas length : {}".format(len(data)))
print("datas length(day) : {}".format(int(len(data) / (24 * 4))))
data.head()

datas length : 35040
datas length(day) : 365


,ttime,energy
0,2017-01-01 00:00:00,60500
1,2017-01-01 00:15:00,63250
2,2017-01-01 00:30:00,59000
3,2017-01-01 00:45:00,57500
4,2017-01-01 01:00:00,58250


# 2. Data Preprocessing (데이터 전처리)
- col : time / row : days 의 형식으로 dataframe 구성

In [62]:
days = [day.split(" ")[0] for day in data['ttime'][::96].tolist()]
df = pd.DataFrame(columns=['day'] + [_ for _ in range(0, 96)])

for day_cnt, day in enumerate(days):
    _ = [day]
    _ += data['energy'][96 * day_cnt:96 * (day_cnt + 1)].tolist()
    
    df.loc[day_cnt] = _
    
df.set_index("day", inplace=True)
df.head()

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
day,,,,,,,,,,,,,,,,,,,,,
2017-01-01,60500,63250,59000,57500,58250,57000,56250,56250,56250,57500,...,70500,71250,67750,60500,59750,61250,60500,61250,59000,58250
2017-01-02,80000,85000,83500,82000,82750,82000,80000,82750,82750,77750,...,97250,95750,90000,82750,77750,76250,75000,75000,75000,71250
2017-01-03,97250,104500,100750,102250,100750,98750,98000,98750,97250,96500,...,127500,123000,113000,102250,100000,97250,98000,93500,92250,90750
2017-01-04,108000,111500,111000,109500,110250,108000,110250,111000,111000,109500,...,124500,112250,99250,90000,88500,87000,87000,83500,85000,82750
2017-01-05,102250,106500,108750,108000,105000,104500,105750,105000,104500,105000,...,116000,112250,108000,101500,93500,90750,86500,84250,85750,85000


# 3. Dimensionality Reduction (차원축소)
- 초반 K 선정을 위한 차원축소

In [109]:
# rule of thumb
K = round(mt.sqrt((len(df) / 2)))

# check similarity (mean_timeseries, special_day_timeseries)
sim_df = pd.DataFrame(columns=['거리', '방향'])
mean_timeseries = df.mean().round().values
for day in df.index:
    timeseries = df.loc[day].values
    dis = euclidean_distance(mean_timeseries, timeseries)
    cos = cosine_similarity(mean_timeseries, timeseries)

    sim_df.loc[day] = [dis, cos]
    
K_idx = list()

sort_sim_df = sim_df.sort_values(by=['거리', '방향'], ascending=[True, False])
sort_sim_df

,거리,방향
2017-04-26,76497.954665,0.998315
2017-04-21,76632.679504,0.998275
2017-06-14,87919.588648,0.997835
2017-06-15,91549.148374,0.998123
2017-09-28,98459.728153,0.997830
...,...,...
2017-01-23,862050.776386,0.990519
2017-01-24,882440.930923,0.992856
2017-02-01,886129.233559,0.984526
2017-12-31,894343.000793,0.955170
